# Crunch count 

In [2]:
import cv2
import mediapipe as mp
import math
import numpy as np

# Initialize MediaPipe
mp_pose = mp.solutions.pose

# Initialize Video Capture
video_path = 'mp4/crunch.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Function to project a point onto a line
def project_point_to_line(point, line):
    x1, y1 = line[0]
    x2, y2 = line[1]
    x0, y0 = point

    dx, dy = x2 - x1, y2 - y1
    if dx == 0 and dy == 0:
        return line[0]  # Avoid division by zero

    t = ((x0 - x1) * dx + (y0 - y1) * dy) / (dx ** 2 + dy ** 2)
    projection_x = x1 + t * dx
    projection_y = y1 + t * dy
    return (int(projection_x), int(projection_y))

# Function to calculate the length of a line
def calculate_line_length(line):
    x1, y1 = line[0]
    x2, y2 = line[1]
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Initialize variables to store the lines and crunch count
line_a = None
line_b = None
line_c = None
crunch_count = 0
stage = "down"  # Initial stage is "up"

# Initialize variables for crunch counting
distance_history = []
MIN_DISTANCE = float('inf')
FRAME_COUNT = 0
MAX_FRAME_COUNT = 20  # Number of frames to calculate the minimum distance over

# Initialize MediaPipe Pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    frame_counter = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_counter += 1

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Pose
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Get the coordinates of keypoints 5, 12, 24, 10, 30, and 32
            point5 = (int(landmarks[5].x * frame.shape[1]), int(landmarks[5].y * frame.shape[0]))
            point10 = (int(landmarks[10].x * frame.shape[1]), int(landmarks[10].y * frame.shape[0]))
            point12 = (int(landmarks[12].x * frame.shape[1]), int(landmarks[12].y * frame.shape[0]))
            point24 = (int(landmarks[24].x * frame.shape[1]), int(landmarks[24].y * frame.shape[0]))
            point30 = (int(landmarks[30].x * frame.shape[1]), int(landmarks[30].y * frame.shape[0]))
            point32 = (int(landmarks[32].x * frame.shape[1]), int(landmarks[32].y * frame.shape[0]))

            # Draw dashed line A (5 to 12)
            if line_a is not None:
                cv2.line(frame, line_a[0], line_a[1], (0, 0, 255), 2, cv2.LINE_AA)

            # Draw dashed line B (12 to 24)
            if line_b is not None:
                cv2.line(frame, line_b[0], line_b[1], (0, 0, 255), 2, cv2.LINE_AA)

            # Create Line C (30 to 32)
            line_c = (point30, point32)

            # Calculate the angle between projection of 10 to Line C
            if line_c is not None:
                projected_point10 = project_point_to_line(point10, line_c)
                cv2.line(frame, projected_point10, point10, (0, 255, 0), 2, cv2.LINE_AA)

                # Update the minimum distance for the first 20 frames
                if FRAME_COUNT < MAX_FRAME_COUNT:
                    distance_10_32 = calculate_line_length((point10, point32))
                    MIN_DISTANCE = min(MIN_DISTANCE, distance_10_32)
                    FRAME_COUNT += 1

                

            # Update the lines for the next frame
            line_a = (point5, point12)
            line_b = (point12, point24)

            # Calculate the current distance between Keypoints 10 and 32
            distance_10_32 = calculate_line_length((point10, point32))
            if distance_10_32 < MIN_DISTANCE *(1-0.15) and stage == "down":
                crunch_count+=1
                stage = "up"
            if distance_10_32 > MIN_DISTANCE *(1-0.15):
                stage="down"     
        # Write the frame to the output video

        # Display current distance and minimum distance on the frame
        cv2.putText(frame, f'Current Distance: {distance_10_32:.2f}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Minimum Distance: {MIN_DISTANCE:.2f}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Crunch Count: {crunch_count}', (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Video with Pose Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release VideoCapture and VideoWriter
cap.release()
cv2.destroyAllWindows()


NameError: name 'core' is not defined